# HW 2: Wrangling

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the Minnesota police use of for data, `./data/mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Is this a concern? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing?
3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [62]:
import pandas as pd
import numpy as np

#Q1.1

# retrieve dataset
data_source = "https://raw.githubusercontent.com/jahleeljackson/DS3001-SP26/refs/heads/main/wrangling/assignment/data/airbnb_hw.csv"
airbnb_df = pd.read_csv(data_source)
airbnb_df.head()

,Host Id,Host Since,Name,Neighbourhood,Property Type,Review Scores Rating (bin),Room Type,Zipcode,Beds,Number of Records,Number Of Reviews,Price,Review Scores Rating
0,5162530,NaN,1 Bedroom in Prime Williamsburg,Brooklyn,Apartment,NaN,Entire home/apt,11249.0,1.0,1,0,145,NaN
1,33134899,NaN,"Sunny, Private room in Bushwick",Brooklyn,Apartment,NaN,Private room,11206.0,1.0,1,1,37,NaN
2,39608626,NaN,Sunny Room in Harlem,Manhattan,Apartment,NaN,Private room,10032.0,1.0,1,1,28,NaN
3,500,6/26/2008,Gorgeous 1 BR with Private Balcony,Manhattan,Apartment,NaN,Entire home/apt,10024.0,3.0,1,0,199,NaN
4,500,6/26/2008,Trendy Times Square Loft,Manhattan,Apartment,95.0,Private room,10036.0,3.0,1,39,549,96.0


In [63]:
## EDA

# view price column dtype
print(airbnb_df.Price.dtype) # object

# instances of each example
print(airbnb_df.Price.value_counts())

# null values
print(airbnb_df.Price.isna().sum()) # none


object
Price
150    1481
100    1207
200    1059
125     889
75      873
       ... 
472       1
454       1
328       1
396       1
291       1
Name: count, Length: 511, dtype: int64
0


In [64]:
# change the data type

# must remove commas first
airbnb_df["Price"] = airbnb_df["Price"].str.replace(",", "")
airbnb_df["Price"] = airbnb_df["Price"].astype("int")

# let's see the datatype
airbnb_df.Price.dtype #int 64


dtype('int64')

In [65]:
airbnb_df["Price"]

,Price
0,145
1,37
2,28
3,199
4,549
...,...
30473,300
30474,125
30475,80
30476,35


In [66]:
#Q1.2
# get dataset
data_file = "https://raw.githubusercontent.com/jahleeljackson/DS3001-SP26/refs/heads/main/wrangling/assignment/data/mn_police_use_of_force.csv"
mn_force_df = pd.read_csv(data_file)
mn_force_df.head()

,response_datetime,problem,is_911_call,primary_offense,subject_injury,force_type,force_type_action,race,sex,age,type_resistance,precinct,neighborhood
0,2016/01/01 00:47:36,Assault in Progress,Yes,DASLT1,NaN,Bodily Force,Body Weight to Pin,Black,Male,20.0,Tensed,1,Downtown East
1,2016/01/01 02:19:34,Fight,No,DISCON,NaN,Chemical Irritant,Personal Mace,Black,Female,27.0,Verbal Non-Compliance,1,Downtown West
2,2016/01/01 02:19:34,Fight,No,DISCON,NaN,Chemical Irritant,Personal Mace,White,Female,23.0,Verbal Non-Compliance,1,Downtown West
3,2016/01/01 02:28:48,Fight,No,PRIORI,NaN,Chemical Irritant,Crowd Control Mace,Black,Male,20.0,Commission of Crime,1,Downtown West
4,2016/01/01 02:28:48,Fight,No,PRIORI,NaN,Chemical Irritant,Crowd Control Mace,Black,Male,20.0,Commission of Crime,1,Downtown West


In [86]:
# view column of interest value counts
print("Yes count: ", (mn_force_df.subject_injury == "Yes").sum())
print("No count: ", (mn_force_df.subject_injury == "No").sum())
print("Null count: ", mn_force_df.subject_injury.isna().sum())

nulls_percentage = mn_force_df.subject_injury.isna().sum() / mn_force_df.subject_injury.shape
print("% of null instances: ", (nulls_percentage * 100), "%")

# mn_force_df[["subject_injury", "force_type"]]
mn_force_df.force_type.value_counts().iloc[0] # == "Bodily force"

# As we can see, most of the force type used is bodily force. This proportion is similar to the NaN values in the subject injury
# The kind of subject injury missing is typically as a result of bodily force.


Yes count:  1631
No count:  1446
Null count:  9848
% of null instances:  [76.1934236] %


np.int64(9430)

In [89]:
#Q1.3

pretrial_data = "wrangling/assignment/data/justice_data.parquet"

pretrial_df = pd.read_parquet(pretrial_data)
pretrial_df.head()



FileNotFoundError: [Errno 2] No such file or directory: 'wrangling/assignment/data/justice_data.parquet'

**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks.

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work.
2. Drop any columns that do not contain data.
3. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
4. Clean the Age variable and make a histogram of the ages of the victims.
5. What proportion of victims are male?
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Are sharks more likely to launch unprovoked attacks on men or women? Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Is it more or less likely to be fatal when the victim is male or female? How do you feel about sharks?
9. What proportion of attacks appear to be by white sharks? (Hint: `str.split()` makes a vector of text values into a list of lists, split by spaces.)

**Q3.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?